<div style="border-left: 6px solid #7B61FF; color:white; padding:20px; border-radius:10px; font-family:Arial, sans-serif; text-align:center; font-size:28px; font-weight:bold;">
  🧱 02 – Baseline Model
</div>

<div style="border-left: 6px solid #27ae60; color:white; margin-left:40px; padding:10px; border-radius:10px; font-family:Arial, sans-serif; font-size:24px;">
  <h2 style="margin-top: 0; font-size:24px;">📦 Import Libraries and Define Paths</h2>
</div>

<div style="margin-left:60px; padding:10px;"> 
  <p style="font-size:18px;">This is the initial block of the rare species image classification project.</p>

  <p>In this section, we perform the following tasks:</p>

  <ul style="line-height: 1.6;">
    <li>📁 <strong>Import libraries</strong> for data manipulation (<code>pandas</code>), file paths (<code>pathlib</code>), and image processing (<code>PIL</code>).</li>
    <li>🖼️ <strong>Apply visual styling</strong> using <code>matplotlib</code> and <code>seaborn</code> to ensure clean and consistent plots.</li>
    <li>📂 <strong>Define the main project directories</strong>, including image folders and the metadata CSV file.</li>
    <li>✅ <strong>Automatic path validation</strong> to ensure all required files and directories exist.</li>
  </ul>

  <p>This setup provides a reliable foundation for safely loading and exploring the dataset.</p>
</div>


In [ ]:
# ========================================== 📦 Importar bibliotecas essenciais ==========================================
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import CSVLogger
from pathlib import Path

In [22]:
# ========================================== 📂 Definir caminhos principais do projeto ==========================================
PROJECT_ROOT = Path().resolve().parent

PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
MODELS_DIR = PROJECT_ROOT / 'models'
REPORTS_DIR = PROJECT_ROOT / 'reports'
OUTPUTS_DIR = PROJECT_ROOT / 'output'
LOGS_DIR = OUTPUTS_DIR / 'logs'
PREDICTIONS_DIR = OUTPUTS_DIR / 'predictions'
TRAIN_DIR = PROCESSED_DIR / 'train'
VAL_DIR = PROCESSED_DIR / 'val'
TEST_DIR = PROCESSED_DIR / 'test'

<div style="border-left: 6px solid #27ae60; color:white; margin-left:40px; padding:10px; border-radius:10px; font-family:Arial, sans-serif; font-size:24px;">
  <h2 style="margin-top: 0; font-size:24px;">📦 Define Parameters</h2>
</div>

<div style="margin-left:60px; padding:10px;"> 
  <p>In this section, we define the core parameters that will guide the training process of the model. These include the input image size, batch size, number of training epochs, and the directory structure of the dataset.</p>
  
  <p>Setting these values early ensures consistency across all steps and allows for easier adjustments when experimenting with different model architectures or datasets.</p>
</div>


In [23]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

<div style="border-left: 6px solid #27ae60; color:white; margin-left:40px; padding:10px; border-radius:10px; font-family:Arial, sans-serif; font-size:24px;">
  <h2 style="margin-top: 0; font-size:24px;">📦 Simple CNN Model</h2>
</div>

<div style="margin-left:60px; padding:10px;"> 
  <p>This block defines a basic Convolutional Neural Network (CNN) architecture used as a starting point for image classification.</p>

  <p>The model was built using three convolutional layers followed by max pooling, a flatten layer, a dense layer with ReLU activation, and dropout for regularization. This structure is intentionally simple, serving as a strong baseline for comparing the performance of more complex models.</p>

  <p>All outputs — including the trained model, training logs, accuracy plots, confusion matrix, predictions, and classification reports — were automatically saved in their respective folders: <code>/models</code>, <code>/reports</code>, and <code>/reports/figures</code>.</p>
</div>


In [26]:
def run_cnn_pipeline(train_dir, val_dir, test_dir, model_name="cnn_baseline", image_size=IMAGE_SIZE, batch_size=BATCH_SIZE, epochs=EPOCHS):
    models_dir = MODELS_DIR
    logs_dir = LOGS_DIR
    predictions_dir = PREDICTIONS_DIR
    reports_dir = REPORTS_DIR
    figures_dir = REPORTS_DIR / "figures"
    for d in [models_dir, logs_dir, predictions_dir, figures_dir, reports_dir]:
        d.mkdir(parents=True, exist_ok=True)

    datagen = ImageDataGenerator(rescale=1./255)
    train_generator = datagen.flow_from_directory(train_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical')
    val_generator = datagen.flow_from_directory(val_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical')
    test_generator = datagen.flow_from_directory(test_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', shuffle=False)

    num_classes = train_generator.num_classes

    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

    log_path = logs_dir / f"{model_name}_training_log.csv"
    csv_logger = CSVLogger(log_path, append=False)

    history = model.fit(train_generator, validation_data=val_generator, epochs=epochs, callbacks=[csv_logger])

    model_path = models_dir / f"{model_name}.h5"
    model.save(model_path)

    val_loss, val_acc = model.evaluate(val_generator)

    plt.figure(figsize=(8, 5))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    acc_fig_path = figures_dir / f"{model_name}_accuracy_plot.png"
    plt.savefig(acc_fig_path)
    plt.close()

    predictions = model.predict(test_generator)
    predicted_classes = predictions.argmax(axis=1)
    true_classes = test_generator.classes
    class_indices = test_generator.class_indices
    inv_class_indices = {v: k for k, v in class_indices.items()}
    predicted_labels = [inv_class_indices[i] for i in predicted_classes]
    true_labels = [inv_class_indices[i] for i in true_classes]

    report = classification_report(true_classes, predicted_classes, target_names=list(class_indices.keys()), output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    report_path = reports_dir / f"{model_name}_classification_report.csv"
    report_df.to_csv(report_path)

    cm = confusion_matrix(true_classes, predicted_classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(class_indices.keys()))
    fig, ax = plt.subplots(figsize=(40, 40))
    disp.plot(ax=ax, xticks_rotation='vertical', cmap='Blues')
    cm_path = figures_dir / f"{model_name}_confusion_matrix.png"
    plt.savefig(cm_path)
    plt.close()

    filenames = test_generator.filenames
    results_df = pd.DataFrame({
        "filename": filenames,
        "true_label": true_labels,
        "predicted_label": predicted_labels
    })
    pred_path = predictions_dir / f"{model_name}_predictions.csv"
    results_df.to_csv(pred_path, index=False)

    return {
        "model_path": model_path,
        "log_path": log_path,
        "report_path": report_path,
        "predictions_path": pred_path,
        "accuracy_plot": acc_fig_path,
        "confusion_matrix": cm_path,
        "val_accuracy": val_acc
    }

In [27]:
results = run_cnn_pipeline(
    train_dir=TRAIN_DIR,
    val_dir=VAL_DIR,
    test_dir=TEST_DIR,
    model_name="cnn_baseline"
)

print("📦 Results Summary:\n")
print(f"📁 Model saved at:           {results['model_path']}")
print(f"📄 Training log:             {results['log_path']}")
print(f"📊 Classification report:    {results['report_path']}")
print(f"📑 Predictions CSV:          {results['predictions_path']}")
print(f"📈 Accuracy plot:            {results['accuracy_plot']}")
print(f"📉 Confusion matrix:         {results['confusion_matrix']}")
print(f"✅ Final validation accuracy: {results['val_accuracy']:.2%}")

Found 8627 images belonging to 202 classes.
Found 2157 images belonging to 202 classes.
Found 1199 images belonging to 202 classes.
Epoch 1/10


c:\Users\Tiago Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Tiago Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


270/270 ━━━━━━━━━━━━━━━━━━━━ 51s 184ms/step - accuracy: 0.0242 - loss: 5.2584 - val_accuracy: 0.0529 - val_loss: 5.0883
Epoch 2/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 50s 184ms/step - accuracy: 0.0515 - loss: 5.1168 - val_accuracy: 0.0686 - val_loss: 4.9282
Epoch 3/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 50s 186ms/step - accuracy: 0.0772 - loss: 4.9201 - val_accuracy: 0.0946 - val_loss: 4.7703
Epoch 4/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 50s 183ms/step - accuracy: 0.0897 - loss: 4.7549 - val_accuracy: 0.1062 - val_loss: 89723011968250401048736024231936.0000
Epoch 5/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 50s 186ms/step - accuracy: 0.1174 - loss: 4.5392 - val_accuracy: 0.1178 - val_loss: 4.5812
Epoch 6/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 50s 184ms/step - accuracy: 0.1515 - loss: 4.2832 - val_accuracy: 0.1275 - val_loss: 4.5360
Epoch 7/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 49s 183ms/step - accuracy: 0.1691 - loss: 4.0751 - val_accuracy: 0.1358 - val_loss: 4.4884
Epoch 8/10
270/270 ━━━━━━━━━━━━━━━━━━━━ 49s 181ms/step - accura

68/68 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - accuracy: 0.1427 - loss: 4.6073
38/38 ━━━━━━━━━━━━━━━━━━━━ 25s 651ms/step


c:\Users\Tiago Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Tiago Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Tiago Pedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

📦 Results Summary:

📁 Model saved at:           D:\Repositories\DL_EOLP\models\cnn_baseline.h5
📄 Training log:             D:\Repositories\DL_EOLP\output\logs\cnn_baseline_training_log.csv
📊 Classification report:    D:\Repositories\DL_EOLP\reports\cnn_baseline_classification_report.csv
📑 Predictions CSV:          D:\Repositories\DL_EOLP\output\predictions\cnn_baseline_predictions.csv
📈 Accuracy plot:            D:\Repositories\DL_EOLP\reports\figures\cnn_baseline_accuracy_plot.png
📉 Confusion matrix:         D:\Repositories\DL_EOLP\reports\figures\cnn_baseline_confusion_matrix.png
✅ Final validation accuracy: 14.74%
